> __Purpose:__ This NB tests a CNN in the agglomerative model clustering procedure. With later finetuning. Uses the previously developed PyTorch code

In [1]:
model_str = "CNN"
# Train/test user split: 24/8


In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import torch

np.random.seed(42) 

from moments_engr import *
from agglo_model_clust import *
from cluster_acc_viz_funcs import *
from DNN_FT_funcs import *
from DNN_AMC_funcs import *
from model_classes import *

import os
cwd = os.getcwd()
print("Current Working Directory: ", cwd)


Current Working Directory:  c:\Users\kdmen\Repos\fl-gestures\ELEC573_Proj


In [3]:
path1 = 'C:\\Users\\kdmen\\Box\\Meta_Gesture_2024\\saved_datasets\\filtered_datasets\\$BStand_EMG_df.pkl'

with open(path1, 'rb') as file:
    raw_userdef_data_df = pickle.load(file)  # (204800, 19)

print(raw_userdef_data_df.shape)
raw_userdef_data_df.head()

(204800, 19)


,Participant,Gesture_ID,Gesture_Num,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.362743,-0.801651,-0.383077,-0.195299,-0.203047,-0.464472,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.351553,-0.775334,-0.382545,-0.154773,-0.131977,-0.295204,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.380825,-0.762588,-0.398388,-0.085411,0.017528,-0.205675,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.366795,-0.765464,-0.374423,-0.073225,0.183172,0.009277,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.245578,-0.761283,-0.303976,-0.081947,0.224996,0.103319,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [4]:
# STEP 1: Train a classification model on every single individual user

userdef_df = raw_userdef_data_df.groupby(['Participant', 'Gesture_ID', 'Gesture_Num']).apply(create_feature_vectors)
#output is df with particpant, gesture_ID, gesture_num and feature (holds 80 len vector)
userdef_df = userdef_df.reset_index(drop=True)

#convert Gesture_ID to numerical with new Gesture_Encoded column
label_encoder = LabelEncoder()
userdef_df['Gesture_Encoded'] = label_encoder.fit_transform(userdef_df['Gesture_ID'])

label_encoder2 = LabelEncoder()
userdef_df['Cluster_ID'] = label_encoder2.fit_transform(userdef_df['Participant'])

print(userdef_df.shape)
userdef_df.head()

(3200, 6)


,Participant,Gesture_ID,Gesture_Num,feature,Gesture_Encoded,Cluster_ID
0,P004,close,1,"[[6.079045311063784], [-7.551458873254243], [-...",0,0
1,P004,close,10,"[[5.994789910363704], [-7.978871468164499], [-...",0,0
2,P004,close,2,"[[6.010193380499154], [-7.7063875553339], [-20...",0,0
3,P004,close,3,"[[5.8212078257286874], [-7.463908156909893], [...",0,0
4,P004,close,4,"[[5.974675085061773], [-7.945111601415482], [-...",0,0


In [5]:
all_participants = userdef_df['Participant'].unique()
# Shuffle the participants
np.random.shuffle(all_participants)
# Split into two groups
#train_participants = all_participants[:24]  # First 24 participants
test_participants = all_participants[24:]  # Remaining 8 participants

In [6]:
# Prepare data
data_splits = prepare_data(
    userdef_df, 'feature', 'Gesture_Encoded', 
    all_participants, test_participants, 
    training_trials_per_gesture=8, finetuning_trials_per_gesture=3,
)

In [9]:
features_df = pd.DataFrame(data_splits['train']['feature'])
# Create a new column 'features' that contains all 80 columns as lists
features_df['feature'] = features_df.apply(lambda row: row.tolist(), axis=1)
# Keep only the new combined column
features_df = features_df[['feature']]
# Combine with labels and participant_ids into a single DataFrame
train_df = pd.concat([features_df, pd.Series(data_splits['train']['labels'], name='Gesture_Encoded'), pd.Series(data_splits['train']['participant_ids'], name='participant_ids')], axis=1)
label_encoder = LabelEncoder()
train_df['Cluster_ID'] = label_encoder.fit_transform(train_df['participant_ids'])

features_df = pd.DataFrame(data_splits['intra_subject_test']['feature'])
# Create a new column 'features' that contains all 80 columns as lists
features_df['feature'] = features_df.apply(lambda row: row.tolist(), axis=1)
# Keep only the new combined column
features_df = features_df[['feature']]
# Combine with labels and participant_ids into a single DataFrame
test_df = pd.concat([features_df, pd.Series(data_splits['intra_subject_test']['labels'], name='Gesture_Encoded'), pd.Series(data_splits['intra_subject_test']['participant_ids'], name='participant_ids')], axis=1)
label_encoder = LabelEncoder()
test_df['Cluster_ID'] = label_encoder.fit_transform(test_df['participant_ids'])

# ENTIRELY WITHHOLDING CROSS CLUSTER DATASET (NOVEL TEST SUBJECTS) FOR NOW. 
#test_df
#features_df = pd.DataFrame(data_splits['train']['features'])
## Create a new column 'features' that contains all 80 columns as lists
#features_df['features'] = features_df.apply(lambda row: row.tolist(), axis=1)
## Keep only the new combined column
#features_df = features_df[['features']]
## Combine with labels and participant_ids into a single DataFrame
#train_df = pd.concat([features_df, pd.Series(data_splits['train']['labels'], name='Gesture_Encoded'), pd.Series(data_splits['train']['participant_ids'], name='participant_ids')], axis=1)

In [10]:
data_dfs_dict = {'train':train_df, 'test':test_df}

# Need to update Cluster_ID col at the end of each round, for both dfs

In [11]:
train_df.head()

,feature,Gesture_Encoded,participant_ids,Cluster_ID
0,"[4.0728441780827485, -6.216633410388578, -15.0...",0,P128,22
1,"[4.842766138855771, -7.933026853320965, -18.38...",0,P128,22
2,"[4.343725560839635, -6.20391996380322, -15.560...",0,P128,22
3,"[4.972292120731378, -6.996600730998998, -17.60...",0,P128,22
4,"[4.942130684716379, -6.513267267205743, -16.81...",0,P128,22


In [12]:
merge_log, intra_cluster_performance, cross_cluster_performance, nested_clus_model_dict = DNN_agglo_merge_procedure(data_dfs_dict, model_str, n_splits=2)

Iter 0: 24 Clusters Remaining
Iter 1: 23 Clusters Remaining
Iter 2: 22 Clusters Remaining
Iter 3: 21 Clusters Remaining
Iter 4: 20 Clusters Remaining
Iter 5: 19 Clusters Remaining
Iter 6: 18 Clusters Remaining
Iter 7: 17 Clusters Remaining
Iter 8: 16 Clusters Remaining
Iter 9: 15 Clusters Remaining
Iter 10: 14 Clusters Remaining
Iter 11: 13 Clusters Remaining
Iter 12: 12 Clusters Remaining
Iter 13: 11 Clusters Remaining
Iter 14: 10 Clusters Remaining
Iter 15: 9 Clusters Remaining
Iter 16: 8 Clusters Remaining
Iter 17: 7 Clusters Remaining
Iter 18: 6 Clusters Remaining
Iter 19: 5 Clusters Remaining
Iter 20: 4 Clusters Remaining
Iter 21: 3 Clusters Remaining
Iter 22: 2 Clusters Remaining


# Finetuning Process

In [ ]:
cluster_lst = list(nested_clus_model_dict['Iter18'].keys())
print("Cluster list:")
print(cluster_lst)
print()
print("Cluster 41 model:")
print(nested_clus_model_dict['Iter18'][41])

In [19]:
# EVALUATION

# I would need to 
## 1) know which pids are in each cluster, at each merge iteration, 
## 2) bissect the datasets accordingly, 
## 3) create the trainloaders

# And idk from which merge iteration I would report the results from...
## Obv wouldnt want to show 30 iterations x 2-30 clusters per iter...

In [23]:
# WANT TO COMPARE THE MODEL PERFORMANCE ON THE NOVEL CLIENT PRE AND POST FT
## VARY AMOUNT OF FT: 
## 1) hyperparams (epoch, lr, weight decay, dropout, etc) and 
## 2) how much data from novel_trainFT is used

## Formalizing Procedure Into a Function

In [31]:
# BEST PARAMS FOR GENERIC MODEL
#lr=0.0001, bs=32, num_ep=50, adam, wd=0.0001, dr=0.3, num_conv=3, 32/64/128, k=5, s=2, m=1, bn=True
config = {
    "learning_rate": 0.0001,
    "batch_size": 32,
    "num_epochs": 50,
    "optimizer": "adam",
    "weight_decay": 1e-4,
    "dropout_rate": 0.3,
    "num_conv_layers": 3,
    "conv_layer_sizes": [32, 64, 128], 
    "kernel_size": 5,
    "stride": 2,
    "padding": 1,
    "maxpool": 1,
    "use_batchnorm": True
}

#full_path = os.path.join(cwd, 'ELEC573_Proj', 'models', 'generic_CNN_model.pth')
full_path = os.path.join(cwd, 'models', 'generic_CNN_model.pth')
print("Full Path:", full_path)

# Load the pretrained model
pretrained_generic_CNN_model = CNNModel_3layer(config, input_dim=80, num_classes=10)
pretrained_generic_CNN_model.load_state_dict(torch.load(full_path))

Full Path: c:\Users\kdmen\Repos\fl-gestures\ELEC573_Proj\models\generic_CNN_model.pth


<All keys matched successfully>

In [ ]:
cluster_lst = list(nested_clus_model_dict['Iter18'].keys())
print("Cluster list:")
print(cluster_lst)
print()
print("Cluster 41 model:")
print(nested_clus_model_dict['Iter18'][41])

In [51]:
# Prepare data
one_trial_data_splits = prepare_data(
    userdef_df, 'feature', 'Gesture_Encoded', 
    all_participants, test_participants, 
    training_trials_per_gesture=1, finetuning_trials_per_gesture=1,
)

five_trial_data_splits = prepare_data(
    userdef_df, 'feature', 'Gesture_Encoded', 
    all_participants, test_participants, 
    training_trials_per_gesture=5, finetuning_trials_per_gesture=5,
)

In [52]:
train_pids = np.unique(data_splits['train']['participant_ids'])
novel_participant_ft_data = data_splits['novel_trainFT']
novel_participant_test_data = data_splits['cross_subject_test']
novel_pids = np.unique(data_splits['novel_trainFT']['participant_ids'])
num_local_training_epochs = 60
num_ft_epochs = 50
ft_lr = 0.001

novel_pid_res_dict = {}

for pid_count, pid in enumerate(novel_pids):
    print(f"PID {pid}, {pid_count}/{len(novel_pids)}")
    novel_pid_res_dict[pid] = {}

    # Create the testloader by segmenting out this specific pid
    # Filter based on participant_id
    indices = [i for i, datasplit_pid in enumerate(novel_participant_ft_data['participant_ids']) if datasplit_pid == pid]
    ############## Novel Participant Finetuning Dataset ##############
    ft_dataset = GestureDataset([novel_participant_ft_data['feature'][i] for i in indices], [novel_participant_ft_data['labels'][i] for i in indices])
    ft_loader = DataLoader(ft_dataset, batch_size=config["batch_size"], shuffle=True)
    ############## Novel Participant Intra Testing Dataset ##############
    indices = [i for i, datasplit_pid in enumerate(novel_participant_test_data['participant_ids']) if datasplit_pid == pid]
    intra_test_dataset = GestureDataset([novel_participant_test_data['feature'][i] for i in indices], [novel_participant_test_data['labels'][i] for i in indices])
    intra_test_loader = DataLoader(intra_test_dataset, batch_size=config["batch_size"], shuffle=True)
    ############## Novel Participant Cross Testing Dataset ##############
    indices = [i for i, datasplit_pid in enumerate(novel_participant_test_data['participant_ids']) if datasplit_pid != pid]
    cross_test_dataset = GestureDataset([novel_participant_test_data['feature'][i] for i in indices], [novel_participant_test_data['labels'][i] for i in indices])
    cross_test_loader = DataLoader(intra_test_dataset, batch_size=config["batch_size"], shuffle=True)
    ############## One Trial Cluster Assignment Dataset ##############
    indices = [i for i, datasplit_pid in enumerate(one_trial_data_splits['participant_ids']) if datasplit_pid == pid]
    clust_asgn_dataset = GestureDataset([one_trial_data_splits['feature'][i] for i in indices], [one_trial_data_splits['labels'][i] for i in indices])
    clust_asgn_loader = DataLoader(clust_asgn_dataset, batch_size=config["batch_size"], shuffle=True)

    # 1) Train a local CNN model
    local_model = CNNModel_3layer(config, input_dim=80, num_classes=10)

    local_results = main_training_pipeline(data_splits=None, train_intra_cross_loaders=[ft_loader, intra_test_loader, cross_test_loader],
                        all_participants=train_pids, test_participants=novel_pids, model_type=local_model,  
                        num_epochs=num_local_training_epochs, config=config)
    # This is kind of repeated but whatever
    local_clus_res = evaluate_model(local_model, intra_test_loader)
    novel_pid_res_dict[pid]["local_acc"] = local_clus_res["accuracy"]

    # 2) Have the pretrained CNN model from the best cluster do inference
    #   - Have all cluster models do inference and compare assign to whichever cluster gives best results
    #pretrained_generic_CNN_model

    # Apply all the cluster models at the chosen iteration on the given participant data
    ## Record that cluster's performance (all cluster's performances?... Ideally)

    clus_model_res_dict = {}
    for clus_id in cluster_lst:
        clus_model = nested_clus_model_dict['Iter18'][clus_id]
        clus_res = evaluate_model(clus_model, clust_asgn_loader) 
        clus_acc = clus_res["accuracy"]
        clus_model_res_dict[clus_id] = clus_acc
        
    # Assign participant to highest scoring cluster
    # Find the key with the highest accuracy
    max_key = max(clus_model_res_dict, key=clus_model_res_dict.get)
    max_value = clus_model_res_dict[max_key]
    print(f"The highest accuracy is {max_value} and it is associated with the key {max_key}.")
    print("Full cluster assignment results dict:")
    print(clus_model_res_dict)

    original_cluster_model = nested_clus_model_dict['Iter18'][max_key]

    # Have the pretrained CNN model from the best cluster do inference
    pretrained_clus_res = evaluate_model(original_cluster_model, intra_test_loader)
    novel_pid_res_dict[pid]["pretrained_acc"] = pretrained_clus_res["accuracy"]

    # 3) FT the above model (or all??) pretrained CNN model on the participant
    ft_model, original_cluster_model, train_loss_log, test_loss_log = fine_tune_model(
        original_cluster_model, ft_loader, intra_test_loader, num_epochs=num_ft_epochs, lr=ft_lr)
    ft_clus_res = evaluate_model(ft_model, intra_test_loader)
    novel_pid_res_dict[pid]["ft_acc"] = ft_clus_res["accuracy"]


PID P102, 0/8


KeyError: 'participant_ids'

In [ ]:
local_acc_data = [] 
pretrained_acc_data = [] 
ft_acc_data = [] 
# Collecting data from the dictionary 
for pid in novel_pid_res_dict: 
    local_acc_data.extend(novel_pid_res_dict[pid]['local_acc']) 
    pretrained_acc_data.extend(novel_pid_res_dict[pid]['pretrained_acc']) 
    ft_acc_data.extend(novel_pid_res_dict[pid]['ft_acc']) 
# Plotting box plots 
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5)) 
axes[0].boxplot(local_acc_data) 
axes[0].set_title('Local Accuracy') 
axes[0].set_ylabel('Accuracy') 
axes[1].boxplot(pretrained_acc_data) 
axes[1].set_title('Pretrained Accuracy') 
axes[1].set_ylabel('Accuracy') 
axes[2].boxplot(ft_acc_data) 
axes[2].set_title('Fine-Tuned Accuracy') 
axes[2].set_ylabel('Accuracy') 
plt.tight_layout() 
plt.show()

# Logging / Reporting / Plotting
> Really ought to log results somehow...